# Experiments with the WaterberryFarm classes

In [ ]:
# allow imports from the main source directory
import sys
sys.path.append('..')

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.path import Path
from matplotlib import animation
from functools import partial
import numpy as np
import math
import bz2
import unittest
import timeit
import pathlib 
import pickle
import copy
import logging
# logging.basicConfig(level=logging.WARNING)
logging.basicConfig(level=logging.INFO)
logging.getLogger().setLevel(logging.INFO)

from Environment import Environment, EpidemicSpreadEnvironment, DissipationModelEnvironment, PrecalculatedEnvironment
from InformationModel import StoredObservationIM, GaussianProcessScalarFieldIM, DiskEstimateScalarFieldIM, im_score, im_score_weighted

from WaterberryFarm import WBF_IM_DiskEstimator, create_wbfe, WBF_Score_WeightedAsymmetric


## Verifying that the mask is working correctly.

In [ ]:
wbf, wbfe, savedir = create_wbfe(False, None, typename = "Waterberry")
#wbf, wbfe = load_precalculated_wbfe()
#wbfe.visualize()
#logging.info("Visualize done, proceed to animate")
# This is working at approximately 2 sec per frame for the environment progress. Not very fast, but it should be roughly ok. 
# anim = wbfe.animate_environment()
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2)
ax1.imshow(wbfe.my_strawberry_mask.T, origin="lower")
ax2.imshow(wbfe.my_tomato_mask.T, origin="lower")
ax3.imshow(wbfe.my_soil_mask.T, origin="lower")
wbfe.geometry.visualize(ax4)

plt.show()

## Put everything together, measure the score for a set of observations


In [ ]:
wbf, wbfe, savedir = create_wbfe(False, None, typename = "Waterberry")
# observation locations
# for waterberry
# locations = [[10, 10], [500, 500], [1500, 1500]]
# for miniberry
locations = [[10, 10], [30, 30], [50, 50]]
observations = []
wbfim = WBF_IM_DiskEstimator(wbf.width, wbf.height)
for location in locations:
    x = location[0]
    y = location[1]
    value_tylcv = wbfe.tylcv.value[x, y]
    value_ccr = wbfe.ccr.value[x, y]
    value_soil = wbfe.soil.value[x, y]
    observation = {}
    observation["TYLCV"] = {wbfim.X : x, wbfim.Y : y, wbfim.VALUE : value_tylcv}
    observation["CCR"] = {wbfim.X : x, wbfim.Y : y, wbfim.VALUE : value_ccr}
    observation["Soil"] = {wbfim.X : x, wbfim.Y : y, wbfim.VALUE : value_soil}
    observations.append(observation)
for observation in observations:
    wbfim.add_observation(observation)
logging.info("waterberry information model: proceed(1)")
wbfim.proceed(1)
logging.info("done waterberry information model: proceed(1)")
logging.info("starting to calculate score for waterberry information model")
scorecode = WBF_Score_WeightedAsymmetric()
score = scorecode.score(wbfe, wbfim)
logging.info(f"Waterberry information model score: {score}")
# now visualize the environment and the information model
fig, axes = plt.subplots(2,4)        
wbfe.visualize(fig, axes[0,0], axes[0,1], axes[0,2], axes[0,3])

image_im_tylcv = axes[1,1].imshow(wbfim.im_tylcv.value.T, vmin=0, vmax=1, origin="lower")
axes[1,1].set_title("TYLCV im")
image_im_ccr = axes[1,2].imshow(wbfim.im_ccr.value.T, vmin=0, vmax=1, origin="lower")
axes[1,2].set_title("CCR im")
image_im_ccr = axes[1,3].imshow(wbfim.im_soil.value.T, vmin=0, vmax=1, origin="lower")
axes[1,3].set_title("SOIL im")
plt.show()